In [1]:
import ROOT
%jsroot on
ROOT.PyConfig.DisableRootLogon = True
ROOT.PyConfig.IgnoreCommandLineOptions = False
import numpy as np
 
from __future__ import print_function
from ROOT import TCanvas, TGraph,gROOT,TFile,TObject
from math import sin
from array import array
import ROOT as r
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from array import array
import numpy as np

%matplotlib inline

Welcome to JupyROOT 6.28/06


In [2]:
#path_to_rootf = "~/xlc_sim_outputs/EnergyDep_Test_50keV_42.root"
#path_to_rootf = "~/XLCal2022Data/flight_2022/Barycen.root"
path_to_rootf = "~/xlc_sim_outputs/Test_EnergyDep13_PD30_0_35_negslope.root"
#path_to_rootf = "~/Script_PA_1_RA1.root"
#path_to_rootf = "~/xlc_sim_outputs/flight_like/PD45_rotating.root"
File1 = ROOT.TFile.Open(path_to_rootf ,"read")

tree = File1.Get("tree")

entries = tree.GetEntries()

c = TCanvas("","",500,500)
tree.Draw("asic")
c.Draw('hist')


## Goal:
Take the Asic and Channel number, use these to determine the azimuthal scattering angle.
## Plan:

using: https://app.box.com/file/841618621793

group the asics by column, next this will be able to be broken but by channel 
<br>
using: https://app.box.com/file/841629005697

In [3]:
# Small code to identify which board the event occurs on
def is_even(number):
    if number%2 == 0:
        return True
    else:
        return False


def get_column(asic):
    if asic < 12:
        if is_even(asic):
            return 0
        else:
            return 1
    elif 14<asic < 28:
        if is_even(asic):
            return 2
        else:
            return 3
    elif 31<asic<44:
        if is_even(asic):
            return 4
        else:
            return 5
    elif 47 <= asic:
        if is_even(asic):
            return 6
        else:
            return 7
    elif asic ==12:
        return 8
    elif asic ==44:
        return 9
    else:
        return False

row0 = [0,1,16,17,32,33,48,49]
row1 = [2,3,18,19,34,35,50,51]
row2 = [8,9,24,25,40,41,56,57]
row3 = [10,11,26,27,42,43,58,59]
row4 = [12 ,44, False,False,False,False,False,False]
rowdf = pd.DataFrame(index = [0,1,2,3,4],columns = [0,1,2,3,4,5,6,7])
rowdf.iloc[0] = row0
rowdf.iloc[1] = row1
rowdf.iloc[2] = row2
rowdf.iloc[3] = row3
rowdf.iloc[4] = row4

def get_row(asic):
    index = rowdf.isin([asic]).any(axis=1).idxmax()
    return index

In [4]:
#Map of the channels related to each Asic: for odd Asics this is flipped, hense the extra code in the else statement


pixels = pd.DataFrame(data = {'1':[1,7,11,15,19,23,31,29],'2':[2,0,9,13,17,21,25,30],'3':[4,6,10,14,18,22,26,28],'4':[3,5,8,12,16,20,24,27]} )
display(pixels)

def get_hpixel(column_num,channel):
    pixel_column = pixels[pixels==channel].stack().index.tolist()
    pixel_column = int(pixel_column[0][1])
    if is_even(column_num) and column_num <=8:
        pixel_column = pixel_column
    else:
        if pixel_column == 1:
            pixel_column = 4
        elif pixel_column == 2:
            pixel_column = 3
        elif pixel_column == 3:
            pixel_column = 2
        elif pixel_column == 4:
            pixel_column = 1
    return pixel_column + (4*column_num)

def get_pixel_row(row_num,column_num,channel):
    row = pixels[pixels==channel].stack().index.tolist()
    row = int(row[0][0])
    if is_even(column_num):
        row = row
    else:
        row = 7 - row
    return row + (8*row_num)

dic = {}
for column_num in np.arange(0,10,1):
    for values in pixels.to_numpy():
        for i in np.arange(0,4,1):
            dic[str(column_num)+"__"+str(values[i])] = get_hpixel(column_num,values[i])
#print(dic)
row_dic = {}
for row_num in np.arange(0,10,1):
    for column_num in np.arange(0,10,1):
        for values in pixels.to_numpy().T:
            for i in np.arange(0,8,1):
                row_dic[str(row_num)+"__"+str(column_num)+"__"+str(values[i])] = get_pixel_row(row_num,column_num,values[i])

,1,2,3,4
0,1,2,4,3
1,7,0,6,5
2,11,9,10,8
3,15,13,14,12
4,19,17,18,16
5,23,21,22,20
6,31,25,26,24
7,29,30,28,27


In [5]:

df = r.RDataFrame('tree',path_to_rootf)


horizontal_pixel = np.zeros(tree.GetEntries(),dtype='int')
pixel_row = np.zeros(tree.GetEntries(),dtype='int')
row = np.zeros(tree.GetEntries(),dtype='int')
column = np.zeros(tree.GetEntries(),dtype='int')
channel = np.zeros(tree.GetEntries(),dtype='int')
EventNumn = np.zeros(tree.GetEntries(),dtype='int')
rotationangle = np.zeros(tree.GetEntries(),dtype='float32')
rotatedphi = np.zeros(tree.GetEntries(),dtype='float32')
energies = np.zeros(tree.GetEntries(),dtype='float32')

for event in range(tree.GetEntries()):
    tree.GetEntry(event)
    EventNumn[event] = tree.eventNumber
    column[event] = get_column(tree.asic[0])
    row[event] = get_row(tree.asic[0])
    channel[event] = tree.channel[0]
    rotationangle[event] = tree.rotationangle
    energies[event] = tree.primary_energy
    #horizontal_pixel[event] = get_hpixel(column[event], tree.channel[0])
    horizontal_pixel[event] = int(dic[str(column[event])+"__"+str(channel[event])])
    pixel_row[event] = int(row_dic[str(row[event])+"__"+str(column[event])+"__"+str(channel[event])])
EventNumn = EventNumn
column = column.astype('int')
channel = channel.astype('int')
horizontal_pixel = horizontal_pixel
rotation_angle = rotationangle.astype('float32')
energy = energies.astype('float32')

d = pd.DataFrame()
#d['column'] = column
#d['channel'] = channel
d['EventNumber'] = EventNumn
d['horizontal_pixel'] = horizontal_pixel
d['pixel_row'] = pixel_row
d['rotationangle'] = rotationangle
d['energy'] = energies
data = {key: d[key].values for key in ['EventNumber','horizontal_pixel','pixel_row','rotationangle','energy']}
rdf = r.RDF.FromNumpy(data)

opt = ROOT.RDF.RSnapshotOptions()

opt.fMode = 'update'
opt.fOverwriteIfExists=True
#rdf.Snapshot("tree2","~/XLCal2022Data/flight_2022/Barycen.root","",opt)

rdf.Snapshot("tree2",path_to_rootf,"",opt)
d1 = rdf.Display()
d1.Print()

+-----+-------------+----------+------------------+-----------+---------------+
| Row | EventNumber | energy   | horizontal_pixel | pixel_row | rotationangle | 
+-----+-------------+----------+------------------+-----------+---------------+
| 0   | 316         | 23.2869f | 14               | 14        | 1.00000f      | 
+-----+-------------+----------+------------------+-----------+---------------+
| 1   | 317         | 52.6501f | 27               | 29        | 1.00000f      | 
+-----+-------------+----------+------------------+-----------+---------------+
| 2   | 319         | 18.5743f | 17               | 8         | 1.00000f      | 
+-----+-------------+----------+------------------+-----------+---------------+
| 3   | 321         | 23.9868f | 40               | 37        | 1.00000f      | 
+-----+-------------+----------+------------------+-----------+---------------+
| 4   | 323         | 31.4264f | 39               | 39        | 1.00000f      | 
+-----+-------------+----------+--

In [6]:
rot_ang = rdf.Histo1D('energy')

c = TCanvas("","",500,500)
rot_ang.Draw('h')
rot_ang.SetTitle('Energy Spectrum')
rot_ang.GetXaxis().SetRangeUser(10,90)
rot_ang.GetXaxis().SetTitle("Energy [keV]")
rot_ang.GetYaxis().SetTitle("Counts")
c.SetLeftMargin(0.15)
c.Draw('Hist')
c.SaveAs('~/xlc_sim_outputs/notebook_outputs/Energy_spectrum.png','png')

Info in <TCanvas::Print>: png file /home/jacob/xlc_sim_outputs/notebook_outputs/Energy_spectrum.png has been created
